###  数据集介绍

MNIST 包括6万张28x28的训练样本，1万张测试样本，很多教程都会对它”下手”几乎成为一个 “典范”，可以说它就是计算机视觉里面的Hello World。所以我们这里也会使用MNIST来进行实战。

前面在介绍卷积神经网络的时候说到过LeNet-5，LeNet-5之所以强大就是因为在当时的环境下将MNIST数据的识别率提高到了99%，这里我们也自己从头搭建一个卷积神经网络，也达到99%的准确率


### 手写数字识别

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms

#### hyper parameters

In [5]:
BATCH_SIZE=512 # 大概需要2G的显存
EPOCHS=20 # 总共训练批次
DEVICE=torch.device("cuda" if torch.cuda.is_available() else "cpu")# 让torch判断是否使用GPU，建议使用GPU环境，因为会快很多

因为Pytorch里面包含了MNIST的数据集，所以我们这里直接使用即可。 如果第一次执行会生成data文件夹，并且需要一些时间下载，如果以前下载过就不会再次下载了

由于官方已经实现了dataset，所以这里可以直接使用DataLoader来对数据进行读取

In [7]:
train_loader=torch.utils.data.DataLoader(
        datasets.MNIST('data',train=True,download=False,
                      transform=transforms.Compose([
                          transforms.ToTensor(),
                          transforms.Normalize((0.1307,), (0.3081,))
                      ])),
        batch_size=BATCH_SIZE,shuffle=True)

#### 测试集

In [8]:
test_loader=torch.utils.data.DataLoader(
        datasets.MNIST('data',train=False,transform=transforms.Compose([
            transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))
        ])),
        batch_size=BATCH_SIZE,shuffle=True)

#### 定义网络

网络包含两个卷积层，conv1和conv2，然后紧接着两个线性层作为输出，最后输出10个维度，这10个维度我们作为0-9的标识来确定识别出的是那个数字

In [13]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # input 1,28*28
        self.conv1=nn.Conv2d(1,10,5)  # 10,24*24
        self.conv2=nn.Conv2d(10,20,3) # 20,10*10
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x) # 24
        out = F.max_pool2d(F.relu(out),2,2) # 12
        out = self.conv2(out) # 10
        out = F.relu(out)
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out

#### 实例化一个网络，实例化后用.to方法将网络移动到GPU

优化器用Adam

In [14]:
model = ConvNet().to(DEVICE)
optimizer = optim.Adam(model.parameters())

定义一个训练的函数，将训练的所有操作都封装到这个函数中

In [22]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

封装测试函数

In [16]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

#### 开始训练

In [23]:
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.140354
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.134748
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.061903

Test set: Average loss: 0.0647, Accuracy: 9806/10000 (98%)

Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.060935
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.049801
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.036992

Test set: Average loss: 0.0593, Accuracy: 9805/10000 (98%)

Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.035149
Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.020280
Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.047812

Test set: Average loss: 0.0443, Accuracy: 9862/10000 (99%)

Train Epoch: 4 [14848/60000 (25%)]	Loss: 0.053731
Train Epoch: 4 [30208/60000 (50%)]	Loss: 0.024771
Train Epoch: 4 [45568/60000 (75%)]	Loss: 0.036523

Test set: Average loss: 0.0470, Accuracy: 9845/10000 (98%)

Train Epoch: 5 [14848/60000 (25%)]	Loss: 0.025924
Train Epoch: 5 [30208/60000 (50%)]	Loss: 0.026888
Train Epoch: 5 [45568/60000 (75%)]	Loss: 0.026661

Test 